In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/3.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input = df_input[df_input['Vehicle Url'] != ''].reset_index(drop=True)
df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：150



,No,Type,Part Number,Brand,Description,Application Summary,Warning,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Air Intake Hose,696-428,Dorman - OE Solutions,Engine Air Intake Hose,"Chevrolet Colorado 2022-17, GMC Canyon 2022-17",,84165133;84535596;84548065,,https://www.dormanproducts.com/p-153694-696-42...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Centerline Length"": ""5 in""}, ""1"": {""Cl..."
1,2,Camshaft Position Sensor,907-866,Dorman - OE Solutions,Magnetic Camshaft Position Sensor,"Lexus 2010-97, Toyota 2009-04",,9091905036;90919A5002,,https://www.dormanproducts.com/p-126460-907-86...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Black""}, ""1"": {""Connector Gen..."
2,3,Catalytic Converter with Integrated Exhaust Ma...,673-0181,Dorman - OE Solutions,Manifold Converter - CARB Compliant,Toyota Sienna 2001,NOTE-CARB Converters Are Engine Family Specifi...,2505220020;2505220030;2505220040;2505220080;25...,,https://www.dormanproducts.com/p-146184-673-01...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Air Tube Adaptable"": ""No""}, ""1"": {""Air..."
3,4,Coolant Thermostat Housing,902-5860,Dorman - OE Solutions,Engine Coolant Thermostat Housing,"Lexus 2008-99, Toyota 2010-00",,163230A020;163230A021;1632320020,,https://www.dormanproducts.com/p-104514-902-58...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Air Vent (Bleeder Valve)"": ""No""}, ""1"":..."
4,5,Coolant Thermostat Housing,902-5926,Dorman - OE Solutions,Engine Coolant Thermostat Housing,"Lexus 2008-99, Toyota 2010-00",,163210A030;1632120030,,https://www.dormanproducts.com/p-104472-902-59...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Air Vent (Bleeder Valve)"": ""No""}, ""1"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,161,Timing Cover,635-811,Dorman - OE Solutions,Timing Cover Kit,"Hyundai Elantra 1994-92, Hyundai Sonata 1994-92",,21360-33001,,https://www.dormanproducts.com/p-50582-635-811...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Black""}, ""1"": {""Grade Type"": ..."
146,162,Timing Cover,635-813,Dorman - OE Solutions,Outer Timing Cover Kit,"Dodge Ram 2500 1998-94, Dodge Ram 3500 1998-94",,3923898;4761240,,https://www.dormanproducts.com/p-137282-635-81...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Black""}, ""1"": {""Depth"": ""1.01..."
147,163,Timing Tab,635-550,Dorman - OE Solutions,Timing Cover Timing Mark,"Chevrolet 1990-66, GMC 1990-68",,,,https://www.dormanproducts.com/p-84483-635-550...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Grade Type"": ""Standard""}, ""1"": {""Indic..."
148,164,Water Pump Inlet Tube,626-304,Dorman - OE Solutions,Heater Hose Assembly - Includes Seals,"Chrysler 2008-01, Dodge 2008-01",,4781608AA;4781608AB;4781608-AC,,https://www.dormanproducts.com/p-48800-626-304...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color/Finish"": ""Black/Enamel Coated""},..."


In [4]:
crawler_index, crawler_series = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                                     1
 Type                                                     Air Intake Hose
 Part Number                                                      696-428
 Brand                                              Dorman - OE Solutions
 Description                                       Engine Air Intake Hose
 Application Summary       Chevrolet Colorado 2022-17, GMC Canyon 2022-17
 Warning                                                                 
 OEM                                           84165133;84535596;84548065
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-153694-696-42...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Centerline Length": "5 in"}, "1": {"Cl...
 Name: 0, dtype: object)

In [5]:
import time

list_dict_correct_temp, page = [], 0
while True:

    crawler_retry = 0
    while True:
        crawler_retry += 1
        print(page + 1, crawler_retry)
        try:
            url_request = f'''{crawler_series['Vehicle Url']}&num=50&start={page * 50}'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath('//div[@class="detail-app-pager"]'):
                    if page == 0:
                        with open('./vehicle.html', 'w', encoding='UTF-8') as file:
                            file.write(str(soup))
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = =  = = =  = = =

    next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
    if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
        break

    # = = =  = = =  = = =

    list_text, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

    # = = =  = = =  = = =

    list_dict_correct_temp_temp = [{'Vehicle Url': crawler_series['Vehicle Url'],
                                    'Page': page + 1,
                                    'Row': i + 1} for i in range(len(list_tr))]

    for i, tr in enumerate(list_tr):
        for j, text in enumerate(list_text):
            list_dict_correct_temp_temp[i][text] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

    list_dict_correct_temp.extend(list_dict_correct_temp_temp)

    # = = =  = = =  = = =

    if not next_:
        break

    # = = =  = = =  = = =

    page += 1

list_dict_correct_temp

1 1


[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 1,
  'Year': '2022',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 2,
  'Year': '2022',
  'Make': 'GMC',
  'Model': 'Canyon',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 3,
  'Year': '2021',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dorman

In [6]:
list_dict_correct_temp

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 1,
  'Year': '2022',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 2,
  'Year': '2022',
  'Make': 'GMC',
  'Model': 'Canyon',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 3,
  'Year': '2021',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dorman

In [7]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 1,
  'Year': '2022',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 2,
  'Year': '2022',
  'Make': 'GMC',
  'Model': 'Canyon',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=153694&PartType=Air%20Intake%20Hose',
  'Page': 1,
  'Row': 3,
  'Year': '2021',
  'Make': 'Chevrolet',
  'Model': 'Colorado',
  'Configuration': 'V6 222 3.6L',
  'Positions': 'Air Cleaner To Engine',
  'Application Notes': 'Factory Installed'},
 {'Vehicle Url': 'https://www.dorman

In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                                     1
 Type                                                     Air Intake Hose
 Part Number                                                      696-428
 Brand                                              Dorman - OE Solutions
 Description                                       Engine Air Intake Hose
 Application Summary       Chevrolet Colorado 2022-17, GMC Canyon 2022-17
 Warning                                                                 
 OEM                                           84165133;84535596;84548065
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-153694-696-42...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Centerline Length": "5 in"}, "1": {"Cl...
 Request_Url            https://www.do

In [9]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Vehicle Url', 'Page', 'Row'],
                                                             ascending=[True, True, True],
                                                             ignore_index=True)
    df_correct['Year'] = df_correct['Year'].astype(int)
    df_correct.to_excel('./test_compatibility.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,https://www.dormanproducts.com/itemdetailapp.a...,1,1,2022,Chevrolet,Colorado,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
1,https://www.dormanproducts.com/itemdetailapp.a...,1,2,2022,GMC,Canyon,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
2,https://www.dormanproducts.com/itemdetailapp.a...,1,3,2021,Chevrolet,Colorado,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
3,https://www.dormanproducts.com/itemdetailapp.a...,1,4,2021,GMC,Canyon,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
4,https://www.dormanproducts.com/itemdetailapp.a...,1,5,2020,Chevrolet,Colorado,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
5,https://www.dormanproducts.com/itemdetailapp.a...,1,6,2020,GMC,Canyon,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
6,https://www.dormanproducts.com/itemdetailapp.a...,1,7,2019,Chevrolet,Colorado,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
7,https://www.dormanproducts.com/itemdetailapp.a...,1,8,2019,GMC,Canyon,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
8,https://www.dormanproducts.com/itemdetailapp.a...,1,9,2018,Chevrolet,Colorado,V6 222 3.6L,Air Cleaner To Engine,Factory Installed
9,https://www.dormanproducts.com/itemdetailapp.a...,1,10,2018,GMC,Canyon,V6 222 3.6L,Air Cleaner To Engine,Factory Installed


In [10]:
print('Output Finished !')
print()

Output Finished !



In [11]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_compatibility_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Type,Part Number,Brand,Description,Application Summary,Warning,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification,Request_Url
0,1,Air Intake Hose,696-428,Dorman - OE Solutions,Engine Air Intake Hose,"Chevrolet Colorado 2022-17, GMC Canyon 2022-17",,84165133;84535596;84548065,,https://www.dormanproducts.com/p-153694-696-42...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Centerline Length"": ""5 in""}, ""1"": {""Cl...",https://www.dormanproducts.com/itemdetailapp.a...


In [12]:
print('Done ~')

Done ~
